将图片标签和图片对格式的数据进行训练，结果并不理想，猜测大概是两方面的原因：
1. 标签使用的是英文
2. 单个batch-size中，出现相同标签或图片的概率过大

考虑以上两种原因，做了如下调整：
1. 将标签翻译成对应的英文
2. 将图片的多个标签转换为一段话

In [8]:
import os
import json
from collections import Counter


# 读取json文件内容
def load_json_data(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as f:
        return json.load(f)


# 图片描述提取器
class ImageDescriptionGenerator:
    def __init__(self, label_json_file_path):
        self.tag_translation_map = load_json_data(r'category_map.json')
        self.label_json_data = load_json_data(label_json_file_path)
        self.frames = self.label_json_data['frames']
        self.attributes = self.label_json_data['attributes']

    # 提取标签
    def extract_tags(self):
        tags = []
        objects = self.frames[0]['objects']
        for obj in objects:
            if obj['category'] not in self.tag_translation_map:
                continue

            tag_name = obj['category']
            if obj['category'] in ['traffic sign', 'traffic light']:
                color = obj['attributes'].get('trafficLightColor', 'none')
                tag_name = f"{obj['category']}/{color}"
            tags.append(tag_name)

        return tags

    # 统计标签
    def count_tags(self):
        tag_counter = Counter()
        tags = self.extract_tags()
        for tag in tags:
            tag_counter[tag] += 1
        return tag_counter

    # 格式化标签
    def format_tags(self, tag_counter):
        tag_scale_map = {
            "traffic light": "个",
            "traffic light/green": "个",
            "traffic light/red": "个",
            "traffic light/yellow": "个",

            "traffic sign": "个",
            "traffic sign/green": "个",
            "traffic sign/red": "个",
            "traffic sign/yellow": "个",

            "car": "辆",
            "area/drivable": "个",
            "lane/road curb": "个",
        }

        format_list = []
        for tag, count in tag_counter.items():
            if tag in tag_scale_map:
                format_list.append(f"{count}{tag_scale_map[tag]}{self.tag_translation_map[tag]['name']}")

        return "，".join(format_list)

    # 生成描述
    def generate_description(self):
        base_part = "在这个{weather}的{timeofday}，{scene}中，可以看到："
        base_part = base_part.format(
            weather=self.tag_translation_map["weather/" + self.attributes['weather']]['name'],
            timeofday=self.tag_translation_map["timeofday/" + self.attributes['timeofday']]['name'],
            scene=self.tag_translation_map["scene/" + self.attributes['scene']]['name'],
        )

        tag_counter = self.count_tags()
        parts = self.format_tags(tag_counter)

        return base_part + parts + "。"


# 读取图片字符串名称和数字名称的映射
def get_image_id_map(split):
    image_id_map_filename = "{}_image_id_map.json".format(split)
    with open(image_id_map_filename, 'r') as f:
        image_id_map = json.load(f)

    return image_id_map


def description_filter(description):
    if len(description) == 0:
        return ""

    if "未知" in description or "其他" in description or "未定义" in description:
        return ""

    if (len(description)) > 50:
        return ""

    return description


def main(src_dir, dst_dir, split):
    # 读取图片字符串名称和数字名称的映射
    image_id_map = get_image_id_map(split)

    text_id = 0
    for dir_path, _, filenames in os.walk(src_dir):
        for i, filename in enumerate(filenames):
            # print("processing: {}, {}".format(i, filename))

            filepath = os.path.join(dir_path, filename)
            image_name = filename.rstrip(".json")
            image_id = image_id_map[image_name]

            # 生成描述
            generator = ImageDescriptionGenerator(filepath)
            description = generator.generate_description()

            # 过滤不期望的描述
            description = description_filter(description)
            if len(description) == 0:
                continue

            print(description)

            # 对图片标签进行拆分
            texts_jsonl_filename = os.path.join(dst_dir, split + "_texts.jsonl")
            text_id = text_id + 1
            with open(texts_jsonl_filename, 'a', encoding='utf-8') as f:
                imageCNClipJson = {
                    "text_id": text_id,
                    "text": description,
                    "image_ids": [image_id],
                }
                json.dump(imageCNClipJson, f, ensure_ascii=False)  # 将字典写入文件
                f.write('\n')  # 写入换行符，表示新的一行


if __name__ == '__main__':
    split_map = [
        {
            "base_name": "train",
            "new_name": "train",
        },
        {
            "base_name": "val",
            "new_name": "valid",
        },
        {
            "base_name": "test",
            "new_name": "test",
        }
    ]
    
    for item in split_map:
        base_dir =  r'E:\playground\ai\datasets\bdd100k\bdd100k_labels\bdd100k\labels\100k'
        src_dir = os.path.join(base_dir, item['base_name'])
        dst_dir = r'E:\playground\ai\datasets\bdd100kLabelsToCNClipDesc'
                
        print(src_dir, dst_dir, item['new_name'])
        
        main(src_dir, dst_dir, item['new_name'])


E:\playground\ai\datasets\bdd100k\bdd100k_labels\bdd100k\labels\100k\train E:\playground\ai\datasets\bdd100kLabelsToCNClipDesc train
在这个晴朗的白天，城市街道中，可以看到：1个绿色交通灯，2辆汽车，1个可行驶区域，3个路缘石。


KeyboardInterrupt: 